## Car Wash Simulation

A car wash simuation where we only have the arrival times of each car and we seen the time they spent in the queue, the time it took for the car washing processing, the time they got free from the car wash and the total time they spent at the car wash. The car wash process is normally distributed with a mean of 15 and std dev of 2. There are 20 cars whose car wash we are processing between 9 AM to 6 PM

In [ ]:
# import sys
# !{sys.executable} -m pip install 

Create random datetimes between two date ranges

In [1]:
from random import randrange
import datetime
# from datetime import timedelta

In [2]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + datetime.timedelta(seconds=random_second)

In [3]:
d1 = datetime.datetime.strptime('1/1/2009 9:00 AM', '%m/%d/%Y %I:%M %p')
d2 = datetime.datetime.strptime('1/1/2009 6:00 PM', '%m/%d/%Y %I:%M %p')


N = 20
arrival_times = []
for i in range(N):
    arrival_times.append(random_date(d1, d2))
    
arrival_times[:5]

[datetime.datetime(2009, 1, 1, 17, 26, 44),
 datetime.datetime(2009, 1, 1, 9, 22, 42),
 datetime.datetime(2009, 1, 1, 16, 1, 19),
 datetime.datetime(2009, 1, 1, 15, 0, 20),
 datetime.datetime(2009, 1, 1, 16, 5, 54)]

Create a dataframe

In [4]:
import pandas as pd
import numpy as np
# pd.to_datetime(arrival_times[0])


df = pd.DataFrame()
df['Cars'] = list(np.arange(N) + 1)
df['Arrival Time'] = arrival_times
df['Arrival Time'] = df['Arrival Time'].sort_values(ascending=True).tolist()

df

,Cars,Arrival Time
0,1,2009-01-01 09:09:26
1,2,2009-01-01 09:22:42
2,3,2009-01-01 10:22:08
3,4,2009-01-01 11:10:55
4,5,2009-01-01 11:13:13
5,6,2009-01-01 11:44:23
6,7,2009-01-01 13:46:05
7,8,2009-01-01 13:58:50
8,9,2009-01-01 14:07:26
9,10,2009-01-01 15:00:20


In [5]:
df.dtypes

Cars                     int64
Arrival Time    datetime64[ns]
dtype: object

Creating a washing process whose time is normally distributed with a mean of mu and stddev of sigma

In [46]:
import math

def washing_process(mu, sigma):
    random_number = np.random.normal(mu, sigma)
    hours = math.floor(random_number/60)
    minutes = math.floor(random_number) if random_number < 60 else (math.floor(random_number) - (hours*60))
    seconds = round(((random_number - math.floor(random_number))*0.6)*100)
    return hours, minutes, seconds

mu = 15
sigma = 2

Functions to change timedelta and datetime.time

In [43]:
def time2timedelta(timeObj):
    return datetime.datetime.combine(datetime.date.min, timeObj) - datetime.datetime.min

def timedelta2time(deltaObj):
    return (datetime.datetime.min + deltaObj).time()

The Simulation

In [60]:
start_times, queue_times, process_times, finish_times, waited_times = [], [], [], [], []

start_times.append(df['Arrival Time'][0].to_pydatetime()) # Arrival time will be the same as start time for the first car
queue_times.append(timedelta2time(datetime.timedelta(seconds=0))) # Adding 0 queuing time for the first car

washTime = washing_process(mu, sigma) # Randomly generate a washing time from anjormal distribution with mean 15 and stddev 2
process_times.append(timedelta2time(datetime.timedelta(hours = washTime[0], minutes = washTime[1], seconds = washTime[2])))

finishTime = start_times[0] + time2timedelta(process_times[0]) # When the car was process finishes
finish_times.append(finishTime)

waited_times.append(timedelta2time(finishTime - start_times[0]))

for index, row in df.iterrows():
    if index == 0:
        continue
    arrivalTime = df['Arrival Time'][index].to_pydatetime()
    if arrivalTime >= finish_times[-1]:
        
        queue_times.append(timedelta2time(datetime.timedelta(seconds=0)))
        start_times.append(df['Arrival Time'][index].to_pydatetime())
        
        washTime = washing_process(mu, sigma)
        process_times.append(timedelta2time(datetime.timedelta(hours = washTime[0], minutes = washTime[1], seconds = washTime[2])))
        
        finishTime = start_times[-1] + time2timedelta(process_times[-1])
        finish_times.append(finishTime)
        
        waited_times.append(timedelta2time(finishTime - arrivalTime))
    else:
        queue_times.append(timedelta2time(finish_times[-1] - arrivalTime))
        start_times.append(finish_times[-1])
        
        washTime = washing_process(mu, sigma)
        process_times.append(timedelta2time(datetime.timedelta(hours = washTime[0], minutes = washTime[1], seconds = washTime[2])))
        
        finishTime = start_times[-1] + time2timedelta(process_times[-1])
        finish_times.append(finishTime)
        
        waited_times.append(timedelta2time(finishTime - arrivalTime))

df['Queue Time'] = queue_times
df['Start Time'] = start_times
df['Process Time'] = process_times
df['Finish Time'] = finish_times
df['Waiting Time'] = waited_times
df      
        
        

,Cars,Arrival Time,Queue Time,Start Time,Process Time,Finish Time,Waiting Time
0,1,2009-01-01 09:09:26,00:00:00,2009-01-01 09:09:26,00:14:52,2009-01-01 09:24:18,00:14:52
1,2,2009-01-01 09:22:42,00:01:36,2009-01-01 09:24:18,00:15:49,2009-01-01 09:40:07,00:17:25
2,3,2009-01-01 10:22:08,00:00:00,2009-01-01 10:22:08,00:10:51,2009-01-01 10:32:59,00:10:51
3,4,2009-01-01 11:10:55,00:00:00,2009-01-01 11:10:55,00:15:01,2009-01-01 11:25:56,00:15:01
4,5,2009-01-01 11:13:13,00:12:43,2009-01-01 11:25:56,00:15:26,2009-01-01 11:41:22,00:28:09
5,6,2009-01-01 11:44:23,00:00:00,2009-01-01 11:44:23,00:16:52,2009-01-01 12:01:15,00:16:52
6,7,2009-01-01 13:46:05,00:00:00,2009-01-01 13:46:05,00:12:45,2009-01-01 13:58:50,00:12:45
7,8,2009-01-01 13:58:50,00:00:00,2009-01-01 13:58:50,00:16:26,2009-01-01 14:15:16,00:16:26
8,9,2009-01-01 14:07:26,00:07:50,2009-01-01 14:15:16,00:15:22,2009-01-01 14:30:38,00:23:12
9,10,2009-01-01 15:00:20,00:00:00,2009-01-01 15:00:20,00:14:08,2009-01-01 15:14:28,00:14:08
